In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from diffusers import AutoencoderKLCogVideoX, CogVideoXImageToVideoPipeline, CogVideoXTransformer3DModel
from diffusers.utils import export_to_video, load_image
from transformers import T5EncoderModel
import numpy as np
model_id = "/datasets/sai/gencam/cogvideox/CogVideoX1.5-5B-I2V/models--THUDM--CogVideoX1.5-5B-I2V/snapshots/46c90528707aebbe69066390b4fe7e7d24c9c2a4"

transformer = CogVideoXTransformer3DModel.from_pretrained(model_id, subfolder="transformer", torch_dtype=torch.bfloat16, low_cpu_mem_usage=True)
text_encoder = T5EncoderModel.from_pretrained(model_id, subfolder="text_encoder", torch_dtype=torch.bfloat16, low_cpu_mem_usage=True)
vae = AutoencoderKLCogVideoX.from_pretrained(model_id, subfolder="vae", torch_dtype=torch.bfloat16, low_cpu_mem_usage=True)

# Create pipeline and run inference
pipe = CogVideoXImageToVideoPipeline.from_pretrained(
    model_id,
    text_encoder=text_encoder,
    transformer=transformer,
    vae=vae,
    torch_dtype=torch.bfloat16,
)


pipe.to(device='cuda')
pipe.enable_sequential_cpu_offload()
pipe.vae.enable_tiling()


/home/tedlasai/miniconda3/envs/gencam/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading pipeline components...: 100%|██████████| 5/5 [00:00<00:00, 22.79it/s]


In [ ]:
path = "/datasets/sai/gencam/FullDataset/Adobe240/lower_fps_frames/IMG_0173/frame_00299.png"

# Load image
image = load_image(path)

prompt = "A person walking."

with torch.no_grad():
    video = pipe(image=image, prompt=prompt, guidance_scale=4, use_dynamic_cfg=True, num_inference_steps=50, num_frames = 17).frames[0]
export_to_video(video, "output.mp4", fps=8)

100%|██████████| 50/50 [05:09<00:00,  6.19s/it]


'output.mp4'